# ETL for CourseData.db
A step-by-step walkthrough

## Preliminaries: Extensions, Imports, and Database Connections

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseData.db
conn = sqlite3.connect('CourseData.db')

## 1. Create Tables from ERD

In [3]:
%%sql

-- Program table
CREATE TABLE PROGRAMS (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT NOT NULL,
    ProgramName TEXT NOT NULL
);

-- Course Catalog table
CREATE TABLE CATALOG_COURSES (
    CourseID INTEGER PRIMARY KEY,
    CatalogYear TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    ProgramID INTEGER,
    CourseTitle TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees TEXT,
    Attributes TEXT,
    Description TEXT
);
CREATE INDEX ix_catalog_courses_alt_key on CATALOG_COURSES(CatalogYear,CatalogID);

-- Instructors table
CREATE TABLE INSTRUCTORS (
    InstructorID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);
CREATE INDEX ix_instructors_name on INSTRUCTORS(Name);

-- Course Offerings table
CREATE TABLE COURSE_OFFERINGS (
    CourseOfferingID INTEGER PRIMARY KEY,
    CourseID INTEGER,
    CatalogID TEXT NOT NULL,
    Term TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    Section TEXT NOT NULL,
    Credits REAL,
    Title TEXT NOT NULL,
    Timecodes TEXT,
    PrimaryInstructorID INTEGER,
    Capacity INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL
);
CREATE INDEX ix_course_offerings_alt_key on COURSE_OFFERINGS(Term,CatalogID,Section);

-- Locations table
CREATE TABLE LOCATIONS (
    LocationID INTEGER PRIMARY KEY,
    LocationCode TEXT NOT NULL
);

-- Course Meetings table
CREATE TABLE COURSE_MEETINGS (
    CourseMeetingID INTEGER PRIMARY KEY,
    CourseOfferingID INTEGER NOT NULL,
    LocationID INTEGER NOT NULL,
    StartDateTime TEXT NOT NULL,
    EndDateTime TEXT NOT NULL
);

-- A conversion table for matching the term to the corresponding catalog year
-- Catalogs are not available for the first several years
CREATE TABLE TERM_CATALOG_YEAR (
    CatalogYear TEXT NOT NULL,
    Term TEXT NOT NULL
);

INSERT INTO TERM_CATALOG_YEAR (Term, CatalogYear) VALUES 
('Fall2017','2017_2018'),('Winter2018','2017_2018'),('Spring2018','2017_2018'),('Summer2018','2017_2018'),('Fall2018','2018_2019'),
('Winter2019','2018_2019'),('Spring2019','2018_2019');

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
7 rows affected.


[]

## 2. Extract data from CSV files
_The following code uses Python to automate the dirty work that you might do in `sqlite3`._

In [4]:
# Catalog Data
catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = 'SourceData/Catalogs/CourseCatalog'+cat_year+'.csv'
    data = pd.read_csv(filepath)
    data['cat_year'] = cat_year
    data.to_sql('IMPORT_CATALOG_COURSES',conn,if_exists='append',index=False)


In [5]:
# Course Offering and Course Meeting Data
terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
         'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 
    
    filepath = 'SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False)

In [6]:
%%sql
-- Record Counts for Catalog Courses
SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOG_COURSES) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOG_COURSES)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
4440,4440


In [7]:
%%sql
-- Record Counts for Course Offerings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_OFFERINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_OFFERINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
15937,15937


In [8]:
%%sql 
-- Record Counts for Catalog Meetings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
284907,284847


__Note: it looks like there are duplicate course meetings in the course_meetings.csv data.__

## 3 & 4. Transform and Load Data Into ERD Tables

In [9]:
%%sql
DELETE FROM INSTRUCTORS;

INSERT INTO INSTRUCTORS (Name)
SELECT DISTINCT primary_instructor 
FROM import_course_offerings 
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%';

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [10]:
%%sql
DELETE FROM PROGRAMS;

INSERT INTO PROGRAMS (ProgramCode,ProgramName)
SELECT DISTINCT program_code,program_name 
FROM IMPORT_CATALOG_COURSES
ORDER BY program_code;

 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

In [11]:
%%sql 
DELETE FROM CATALOG_COURSES;

INSERT INTO CATALOG_COURSES (CatalogYear,ProgramID,CatalogID,CourseTitle,Credits,Prereqs,Coreqs,Fees,Attributes,Description)
SELECT DISTINCT cat_year, ProgramID,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description
FROM IMPORT_CATALOG_COURSES 
    JOIN PROGRAMS ON (program_code = ProgramCode);

 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

In [12]:
%%sql
DELETE FROM COURSE_OFFERINGS;

INSERT INTO COURSE_OFFERINGS (CourseID,Term,CRN,CatalogID,Section,Credits,Title,Timecodes,PrimaryInstructorID,Capacity, Actual, Remaining)
SELECT DISTINCT CourseID,term,crn,catalog_id,section,import_course_offerings.credits,title,timecodes,InstructorID,cap,act,rem 
FROM import_course_offerings 
    LEFT JOIN INSTRUCTORS ON (import_course_offerings.primary_instructor=INSTRUCTORS.Name)
    LEFT JOIN TERM_CATALOG_YEAR USING (Term)
    LEFT JOIN CATALOG_COURSES ON (catalog_id = CatalogID AND CATALOG_COURSES.CatalogYear = TERM_CATALOG_YEAR.CatalogYear)
;

 * sqlite:///CourseData.db
0 rows affected.
15937 rows affected.


[]

In [13]:
%%sql 
INSERT INTO LOCATIONS (LocationCode)
SELECT DISTINCT Location 
FROM import_course_meetings
ORDER BY Location

 * sqlite:///CourseData.db
207 rows affected.


[]

In [14]:
%%sql
DELETE FROM COURSE_MEETINGS;

INSERT INTO COURSE_MEETINGS (CourseOfferingID,LocationID,StartDateTime,EndDateTime)
SELECT DISTINCT COURSE_OFFERINGS.CourseOfferingID, LocationID,`Start`,`End`
FROM import_course_meetings 
    JOIN COURSE_OFFERINGS USING (Term,CRN)
    LEFT JOIN LOCATIONS ON (import_course_meetings.Location = LOCATIONS.LocationCode);

 * sqlite:///CourseData.db
0 rows affected.
284847 rows affected.


[]

## 5. Integrity Checks

### Domain Integrity
The SQLite data types are pretty limited, so there is not much to see here. A few specific value errors were corrected on import. 

### Entity Integrity

In [15]:
%%sql
-- There should be 4440 Catalog Entries, 15937 Course Offerings, and 284847 Course Meetings
SELECT 
    (SELECT Count(*) FROM CATALOG_COURSES) as CatalogCourses,
    (SELECT Count(*) FROM COURSE_OFFERINGS) as CourseOfferings,
    (SELECT Count(*) FROM COURSE_MEETINGS) as CourseMeetings;

 * sqlite:///CourseData.db
Done.


CatalogCourses,CourseOfferings,CourseMeetings
4440,15937,284847


### Relational Integrity

In [20]:
%%sql 
-- A check of COURSE_OFFERINGS --> INSTRUCTORS, COURSE_OFFERINGS --> CATALOG_COURSES, CATALOG_COURSES --> PROGRAMS
SELECT CourseID,CatalogYear,Term,CRN, Section,COURSE_OFFERINGS.CatalogID as CatID, Title,Capacity,Actual,Remaining,substr(Term,-4) as Year
FROM COURSE_OFFERINGS 
    LEFT JOIN INSTRUCTORS ON (COURSE_OFFERINGS.PrimaryInstructorID = INSTRUCTORS.InstructorID)
    LEFT JOIN CATALOG_COURSES USING (CourseID)
    LEFT JOIN PROGRAMS USING (ProgramID)
WHERE Name like '%Huntley%'
ORDER BY Year,Term DESC,CatID,Section;

 * sqlite:///CourseData.db
Done.


CourseID,CatalogYear,Term,CRN,Section,CatID,Title,Capacity,Actual,Remaining,Year
None,None,Fall2014,70369,E,IS 0100,Intro to Information Systems,25,26,-1,2014
None,None,Fall2014,73060,A,IS 0135,Fundamentals of Web Design,25,26,-1,2014
None,None,Fall2014,73061,A,IS 0320,Systems Design and Implementation,25,16,9,2014
None,None,Fall2015,75231,E,IS 0100,Intro to Information Systems,29,28,1,2015
None,None,Fall2015,75246,F,IS 0100,Intro to Information Systems,29,28,1,2015
None,None,Fall2015,76388,A,IS 0135,Fundamentals of Web Design,25,21,4,2015
None,None,Fall2015,76389,A,IS 0320,Systems Design and Implementation,25,13,12,2015
None,None,Spring2016,38780,01,IS 0585,Contemporary Topics: Information Systems and Data,20,15,5,2016
None,None,Spring2016,37253,B,OM 0101,Operations Management,29,28,1,2016
None,None,Spring2016,37254,C,OM 0101,Operations Management,29,29,0,2016


__Note: The Course Catalog data is missing BA 510! The website was updated after the data was scraped in January. Ugh. Should update with new data.__

In [19]:
%%sql
-- A check of the COURSE_MEETINGS --> COURSE_OFFERINGS relationship
SELECT Term, CourseOfferingID, Count(CourseMeetingID)
FROM COURSE_OFFERINGS JOIN COURSE_MEETINGS USING (CourseOfferingID)
WHERE CRN=39006 and Term = 'Spring2019'
GROUP BY CourseOfferingID;

 * sqlite:///CourseData.db
Done.


Term,CourseOfferingID,Count(CourseMeetingID)
Spring2019,13345,9


In [18]:
%%sql
SELECT CourseOfferingID, CourseMeetingID, StartDateTime
FROM COURSE_OFFERINGS JOIN COURSE_MEETINGS USING (CourseOfferingID)
WHERE CRN=39006 AND Term="Spring2019"

 * sqlite:///CourseData.db
Done.


CourseOfferingID,CourseMeetingID,StartDateTime
13345,253416,2019-01-29T18:30:00
13345,253417,2019-02-05T18:30:00
13345,253418,2019-02-12T18:30:00
13345,253419,2019-02-26T18:30:00
13345,253420,2019-03-05T18:30:00
13345,253421,2019-03-12T18:30:00
13345,253422,2019-02-02T09:00:00
13345,253423,2019-02-16T09:00:00
13345,253424,2019-03-09T09:00:00


__There was a bug in the course calendar generator with saturday classes. It's fixed now!__

## 6. Empty out the `IMPORT_` tables to reclaim storage space

In [21]:
%%sql
DELETE FROM IMPORT_CATALOG_COURSES;
DELETE FROM IMPORT_COURSE_OFFERINGS;
DELETE FROM IMPORT_COURSE_MEETINGS;

 * sqlite:///CourseData.db
4440 rows affected.
15937 rows affected.
284907 rows affected.


[]